In [11]:
import pandas as pd

import torch.nn.functional as F
import torch.nn as nn
import torch
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()

from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, BertTokenizer, EncoderDecoderModel

In [4]:
## Data
dataset = load_dataset(*['wmt14', 'de-en'])
# subset = list(range(0, 8))
# dataset['train'] = torch.utils.data.Subset(dataset['train'], subset)
# dataset['validation'] = torch.utils.data.Subset(dataset['validation'], subset)
# dataset['test'] = torch.utils.data.Subset(dataset['test'], subset)

Found cached dataset wmt14 (/home/administrator/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:

### Tokenizers
de_tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
en_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")


In [7]:
df_train = pd.DataFrame.from_dict(dataset['train'])

In [8]:
df_train

,translation
0,"{'de': 'Wiederaufnahme der Sitzungsperiode', '..."
1,"{'de': 'Ich erkläre die am Freitag, dem 17. De..."
2,"{'de': 'Wie Sie feststellen konnten, ist der g..."
3,{'de': 'Im Parlament besteht der Wunsch nach e...
4,{'de': 'Heute möchte ich Sie bitten - das ist ...
...,...
4508780,{'de': 'Das bleibt eine der größten Errungensc...
4508781,{'de': 'Gleichzeitig scheint sich Zumas revolu...
4508782,"{'de': 'In einer Region, wo die älteren Mensch..."
4508783,{'de': 'Drei von zehn Südafrikanern sind jünge...


In [13]:

def num_tokens(x):
    en_len = en_tokenizer(x['en'], padding=True, truncation=True, return_tensors="pt", add_special_tokens=True)['input_ids'].shape[1]
    de_len = de_tokenizer(x['de'], padding=True, truncation=True, return_tensors="pt", add_special_tokens=True)['input_ids'].shape[1]
    return en_len, de_len

df_train['en'], df_train['de'] = zip(*df_train.translation.progress_apply(num_tokens))

  0%|          | 0/4508785 [00:00<?, ?it/s]

In [15]:
lang_from, lang_to = 'en', 'de'

In [20]:
df_train['sum_len'] = df_train[lang_from] + df_train[lang_to]

In [22]:
df_train = df_train[[lang_from, lang_to,'sum_len']].sort_values('sum_len', ascending=False)

In [26]:
df_train['index'] = df_train.index

In [30]:
df_train.to_csv('train.csv', index=False)

In [31]:
!head train.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
en,de,sum_len,index
512,512,1024,3967163
512,512,1024,1505155
512,512,1024,2924173
512,512,1024,2381664
512,512,1024,3964757
512,512,1024,3176806
512,512,1024,2381655
512,512,1024,2391878
512,512,1024,2515103


In [35]:
df_train[(df_train.en<=50) & (df_train.de<=50)]

,en,de,sum_len,index
1869726,50,50,100,1869726
3203990,50,50,100,3203990
1512970,50,50,100,1512970
1525862,50,50,100,1525862
1590117,50,50,100,1590117
...,...,...,...,...
775947,3,3,6,775947
329863,3,3,6,329863
374917,3,3,6,374917
763411,3,3,6,763411


In [ ]:
BS = 2
train_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=BS, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset['validation'], batch_size=BS, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=BS, shuffle=False)